Подключение библиотек. YOLO (You Only Look Once) - это алгоритм машинного обучения, который используется для обнаружения объектов на изображениях и видео.

In [36]:
from ultralytics import YOLO
import numpy as np
import cv2
from scipy.spatial.distance import mahalanobis
     

Загрузка модели и детекция

In [33]:
# Загружаем модель YOLOv8x
model = YOLO("yolov8x.pt")

# Загружаем видео
video = cv2.VideoCapture("test_video_short.mp4")
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Инициализация счетчика ID для трекинга
frame_count = 0
track_ids = {}

# Создание видеозаписи
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('output.mp4', fourcc, 20.0, (width, height))
   

In [34]:
# Цикл по кадрам видео
while(video.isOpened()):
    # Читаем кадр
    ret, frame = video.read()
    if not ret:
        break

    # Выполняем детекцию
    results = model.predict(frame, classes=[0], conf=0.2, iou=0.5)

    # Получаем координаты распознанных людей
    detections = results[0].boxes.xyxy.cpu().numpy().astype(int)

    # Обновляем словарь track_ids
    for det in detections:
        x1, y1, x2, y2 = det
        cur_center_x = (x1 + x2) // 2
        cur_center_y = (y1 + y2) // 2

        # Проверяем, есть ли уже ID для этого человека
        found_id = False
        for id, (prev_center_x, prev_center_y) in track_ids.items():
            # Проверяем расстояние между центрами
            distance = np.sqrt((cur_center_x - prev_center_x)**2 + (cur_center_y - prev_center_y)**2)
            if distance < 50:
                track_ids[id] = (cur_center_x, cur_center_y)
                found_id = True
                break

        # Если ID не найден, создаем новый
        if not found_id:
            new_id = len(track_ids) + 1
            track_ids[new_id] = (cur_center_x, cur_center_y)

    # Рисуем прямоугольники и ID
    for id, (cur_center_x, cur_center_y) in track_ids.items():
        # Находим соответствующую детекцию
        for det in detections:
            x1, y1, x2, y2 = det
            if (x1 < cur_center_x < x2) and (y1 < cur_center_y < y2):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.putText(frame, f'People: {len(track_ids)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 2)
    cv2.imshow("Video", frame)

    # Записываем кадр в файл
    writer.write(frame)

    frame_count += 1

    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождаем ресурсы
video.release()
writer.release()
cv2.destroyAllWindows()


0: 384x640 6 persons, 1769.7ms
Speed: 2.0ms preprocess, 1769.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1757.6ms
Speed: 2.0ms preprocess, 1757.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1735.6ms
Speed: 2.0ms preprocess, 1735.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1738.6ms
Speed: 3.0ms preprocess, 1738.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1734.6ms
Speed: 2.0ms preprocess, 1734.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1740.0ms
Speed: 2.0ms preprocess, 1740.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1736.6ms
Speed: 2.0ms preprocess, 1736.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1734.7ms
Speed: 2.0ms preprocess, 1734.7ms inference, 2.0ms 